In [2]:
import pandas as pd
import psycopg2
import sys
sys.path.insert(0, "../")
# from util.vw_addr_to_coord import GeoCoder
yyyymm = '202305'

In [45]:
# 아우름 데이터뷰 v_a_asset_info 에서 지번 주소를 추출하고 pnu를 맵핑한다
aureumpath = '../asset/rawfile/iasset/v_a_asset_info_202304261449_example.csv'
iassetpath = '../asset/rawfile/iasset/iasset.csv'
assetdf = pd.read_csv(aureumpath, encoding = 'cp949')
addrs = assetdf['INVT_TRGT_LCTN_ADDR']
addrs.to_csv('addrs.csv')
iassetdf = pd.read_csv(iassetpath)
platplcs = iassetdf['PLATPLC']

In [4]:
# Aws connect settings
endpoint = "etlrds.cjezp4ewipgs.ap-northeast-2.rds.amazonaws.com"
port = 5432
database_name = "postgres"
username = "postgres"
password = "dlwltm12#"

# Establish a connection
try:
    connection = psycopg2.connect(
        host=endpoint,
        port=port,
        dbname=database_name,
        user=username,
        password=password
    )
    print("Connected to the PostgreSQL database on AWS RDS")
    cursor = connection.cursor()

except psycopg2.Error as error:
    print("Error connecting to the PostgreSQL database on AWS RDS:", error)

Connected to the PostgreSQL database on AWS RDS


In [ ]:
# Execute a query
# query = "SELECT x.* FROM bddb.address x WHERE platplc like '서울특별시 강서구 등촌동 689%'"
pnus = []
for platplc in platplcs:
# for platplc in ['경기도 용인시 처인구 양지면 양지리 산89-7']:
    print(platplc)
    if (platplc is None) or (platplc == 'NoData') or (type(platplc) != str):
        pnu = 'NoData'        
    else:
        query = f"SELECT x.pnu FROM bddb.address x WHERE platplc like '{platplc}%'"
        cursor.execute(query)
        results = cursor.fetchall()
        try:
            pnu = results[0][0]
            assert len(pnu) == 19
        except:
            bunji = platplc.split(' ')[-1]
            bunji_split = bunji.split('-')
            
            if bunji_split[0][:1] == '산':
                is_mountain = '2'
                bun = bunji_split[0][1:].zfill(4)
                print(bun)
            else:
                is_mountain = '1'
                bun = bunji_split[0].zfill(4) # '0296'     
                
            if len(bunji_split) > 1:
                ji = bunji_split[1].zfill(4) # '0077'
            else:
                ji = '0000'

            sggbjd_str = " ".join(platplc.split(' ')[:-1]) # '경기도 수원시 권선구 서둔동'
            query = f"SELECT x.pnu FROM bddb.address x WHERE platplc like '{sggbjd_str}%'"
            cursor.execute(query)
            results = cursor.fetchall()
            try:
                sggbjd_cd = results[0][0][:10]
                pnu = sggbjd_cd + is_mountain + bun + ji
                assert len(pnu) == 19
            except:
                pnu = 'NoData'
    print(pnu)
    pnus.append(pnu)

In [49]:
iassetdf['PNU'] = pnus
int_cols = ['FUND_CD', 'FUND_NUM', 'ASET_CD', 'INVT_ASET_NM', 'INVT_TRGT_LCTN_ADDR', 'PLATPLC', 'PNU']
col_dict = {
    'FUND_CD': 'fundcd',
    'FUND_NUM': 'fundnum',
    'ASET_CD': 'assetcd',
    'INVT_ASET_NM': 'assetnm',
    'INVT_TRGT_LCTN_ADDR': 'assetaddr',
    'PLATPLC': 'platplc',
    'PNU': 'pnu'
}
iassetdf = iassetdf[int_cols]
iassetdf = iassetdf.rename(columns=col_dict)

In [77]:
iassetdf = iassetdf.astype(str)
iassetdf.to_csv(f'../asset/preprocess/iasset/iasset_{yyyymm}.txt', index = False)

In [54]:
iassetdf

,fundcd,fundnum,assetcd,assetnm,assetaddr,platplc,pnu
0,112001,1호,A112001001,NC백화점(강서),서울특별시 강서구 등촌동 689번지 외 2필지,서울특별시 강서구 등촌동 689,1150010200106890000
1,112001,1호,A112001002,2001아울렛(안양),경기도 안양시 만안구 안양동 627-287,경기도 안양시 만안구 안양동 627-287,4117110100106270287
2,112002,8호,A112002001,포도몰,서울특별시 관악구 신림동 1641-2,서울특별시 관악구 신림동 1641-2,1162010200116410026
3,112003,2호,A112003001,"NH농협생명빌딩 서관(구, 임광빌딩 본관)",서울특별시 서대문구 미근동 267,서울특별시 서대문구 미근동 267,1141010400102670000
4,112003,2호,A112003002,"NH농협생명빌딩 동관(구, 임광빌딩 신관)",서울특별시 서대문구 미근동 257,서울특별시 서대문구 미근동 257,1141010400102570000
...,...,...,...,...,...,...,...
764,R00008,R레지던스,AR00008005,이지스200호(디어스 판교),경기도 성남시 수정구 대왕판교로 815,경기도 성남시 수정구 시흥동 285-2,4113111500102850002
765,R00009,R카이트20호,AR00009001,이천YM물류센터,경기도 이천시 대월면 초지리 273-1,경기도 이천시 대월면 초지리 273-1,4150035025102730001
766,R00010,R글로벌레지던스,AR00010001,Spring Creek Towers,"1255 Granville Payne Ave, Brooklyn, NY",NoData,NoData
767,R00010,R글로벌레지던스,AR00010002,Illini Tower,"409E Chalmers St, Champaign, IL",NoData,NoData


In [1]:
import pandas as pd
import psycopg2
import sys
sys.path.insert(0, "../")
from util.addr_to_coord import GeoCoder
yyyymm = '202305'
iassetdf = pd.read_csv(
    f'../asset/preprocess/iasset/iasset_{yyyymm}.txt',
    header= 0,
    dtype= 'string'
    )

In [2]:
xs = []
ys = []
for i in range(len(iassetdf)):
    platplc = iassetdf.iloc[i]['platplc']
    if (platplc is None) or (platplc == 'NoData') or (type(platplc) != str):
        x, y = 0, 0
    else:
        gc = GeoCoder()
        x, y = gc.AddrToCoord(platplc)
    xs.append(x)
    ys.append(y)

In [6]:
iassetdf['x'] = xs
iassetdf['y'] = ys

In [9]:
iassetdf.to_csv(f'../asset/preprocess/iasset/iasset_{yyyymm}.txt', index = False)